In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import sys, os

In [2]:
from dask.distributed import Client, wait
# client.close()  # Close existing client
client = Client(n_workers=104, threads_per_worker=1, memory_limit=None)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 104
Total threads: 104,Total memory: 0 B
Status: running,Using processes: True
Comm: tcp://127.0.0.1:33459,Workers: 104
Dashboard: /proxy/8787/status,Total threads: 104
Started: Just now,Total memory: 0 B
Comm: tcp://127.0.0.1:39729,Total threads: 1
Dashboard: /proxy/36953/status,Memory: 0 B
Nanny: tcp://127.0.0.1:42391,


# ACCESS-ESM1-5

In [18]:
path = '/g/data/fs38/publications/CMIP6/CMIP/CSIRO/ACCESS-ESM1-5/historical'\
'/r10i1p1f1/day/tas/gn/latest/'
# 'tas_day_ACCESS-ESM1-5_historical_r10i1p1f1_gn_18500101-18991231.nc'

In [31]:
daily_ds = xr.open_mfdataset(path +'*.nc',)['tas']

In [32]:
daily_ds = daily_ds.chunk({'time':-1, 'lat':145//5, 'lon':192//16})

In [ ]:
daily_ds.to_zarr('g/data/w40/ab2313/time_of_emergence/access_tas_daily.zarr')

/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/distributed/client.py:3357: UserWarning: Sending large graph of size 10.27 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


# BEST

BEST is not on a regular grid

In [10]:
daily_ds = xr.open_dataset(
    '/g/data/w40/ab2313/time_of_emergence/best/Complete_TAVG_Daily_EqualArea.nc')

In [4]:
daily_ds

<xarray.Dataset> Size: 1GB
Dimensions:      (map_points: 5498, time: 52077, day_number: 365)
Coordinates:
    longitude    (map_points) float32 22kB ...
    latitude     (map_points) float32 22kB ...
Dimensions without coordinates: map_points, time, day_number
Data variables:
    date_number  (time) float64 417kB ...
    year         (time) float64 417kB ...
    month        (time) float64 417kB ...
    day          (time) float64 417kB ...
    day_of_year  (time) float64 417kB ...
    land_mask    (map_points) float64 44kB ...
    temperature  (time, map_points) float32 1GB ...
    climatology  (day_number, map_points) float32 8MB ...
Attributes:
    Conventions:     Berkeley Earth Internal Convention (based on CF-1.5)
    title:           Native Format Berkeley Earth Surface Temperature Anomaly...
    history:         07-Sep-2022 00:28:01
    institution:     Berkeley Earth Surface Temperature Project
    source_history:  06-Sep-2022 15:50:29
    comment:         This file contains Berkeley Earth surface temperature an...

In [9]:
daily_ds.longitude.values[:30]

array([ -83.57143 ,  -70.71429 ,  -57.857143,  -45.      ,  -32.142857,
        -19.285715,  -90.      ,  -79.411766,  -68.82353 ,  -58.235294,
        -47.64706 ,  -37.058823,  -26.470589,  -15.882353,   47.64706 ,
         58.235294,   90.      ,  100.588234, -105.36585 ,  -96.585365,
        -87.80488 ,  -79.02439 ,  -70.243904,  -61.463413,  -52.682926,
        -43.90244 ,  -35.121952,  -26.341463,  -17.560976,    8.780488],
      dtype=float32)

In [ ]:
import xesmf as xe

# Create source dataset (point data)
source_ds = xr.Dataset(
    {
        "temperature": (["time", "map_points"], daily_ds['temperature'].values)
    },
    coords={
        "lon": (["map_points"], daily_ds['longitude'].values),
        "lat": (["map_points"], daily_ds['latitude'].values),
        "time": daily_ds['time'].values
    }
)

# Create target grid
target_grid = xr.Dataset(
    {
        "lat": (["lat"], np.arange(-90, 90.1, 1.0)),
        "lon": (["lon"], np.arange(-180, 180.1, 1.0))
    }
)

# Regrid
regridder = xe.Regridder(source_ds, target_grid, "nearest_s2d", reuse_weights=False)
temp_on_grid = regridder(source_ds['temperature'])


--------------------------------------------------------------------------
but there are no active ports detected (or Open MPI was unable to use
them).  This is most certainly not what you wanted.  Check your
cables, subnet manager configuration, etc.  The openib BTL will be
ignored for this job.

  Local host: gadi-cpu-spr-0707
--------------------------------------------------------------------------
Exception ignored in: <function WeakSet.__init__.<locals>._remove at 0x14e17b95d510>
Traceback (most recent call last):
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/_weakrefset.py", line 45, in _remove
    self.data.discard(item)
KeyboardInterrupt: 
2025-06-25 18:16:12,909 - distributed.nanny - ERROR - Worker process died unexpectedly
2025-06-25 18:16:12,909 - distributed.nanny - ERROR - Worker process died unexpectedly
Process Dask Worker process (from Nanny):
Process Dask Worker process (from Nanny):
Traceback (most recent call last):
  File "/g/data/h